In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cmx
import matplotlib.colors as colors
from sklearn.metrics import r2_score
from sklearn.neighbors import KNeighborsRegressor
import statsmodels.api as sm
from statsmodels.api import OLS
import scipy as sp
%matplotlib inline
#sets up pandas table display
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns 
sns.despine() 
sns.set_style('whitegrid')
#sets up styles and gives us more plotting options

<Figure size 432x288 with 0 Axes>

In [2]:
from statsmodels.graphics.tsaplots import plot_acf  #自相关图
from statsmodels.tsa.stattools import adfuller as ADF  #平稳性检测
from statsmodels.graphics.tsaplots import plot_pacf    #偏自相关图
from statsmodels.stats.diagnostic import acorr_ljungbox    #白噪声检验
from statsmodels.tsa.arima_model import ARIMA#seaborn 是建立在matplotlib之上的
from statsmodels.tsa.arima_model import ARMA
%matplotlib inline 

%pylab inline

#jupyter中文显示是方框，加入下面两行即可显示中文，若嫌麻烦，也可去网上搜索如何永久显示中文
plt.rcParams['font.sans-serif'] = ['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False #用来正常显示负号
pylab.rcParams['figure.figsize'] = (13, 6)   #设置输出图片大小
#sns.set(color_codes=True) #seaborn设置背景
import matplotlib.pyplot as plt  
plt.rcParams['font.sans-serif'] = [u'SimHei']
plt.rcParams['axes.unicode_minus'] = False

plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']

Populating the interactive namespace from numpy and matplotlib


In [3]:
import pyecharts

In [4]:
from pyecharts import Bar

In [5]:
#半月汇报--两用户端对比
def _doubleweek(data_A,data_I):
    df_Andr=pd.read_csv(data_A)
    df_Ios=pd.read_csv(data_I)
    df=df_Andr.copy()
    df['新增用户']=df_Andr['新增用户']+df_Ios['新增用户']
    df['活跃用户']=df_Andr['活跃用户']+df_Ios['活跃用户']
    df['累计用户']=df_Andr['累计用户']+df_Ios['累计用户']
    df['启动次数']=df_Andr['启动次数']+df_Ios['启动次数']
    
    e = pd.bdate_range('8/5/2018', '6/15/2022')
    df_workday=pd.DataFrame(e.format())
    df_workday.columns =['day']
        
    df_Andr_filter = df_Andr[df_Andr[u'日期'].isin(df_workday['day'])]#工作日安卓新增用户数量
    df_Ios_filter = df_Ios[df_Ios['日期'].isin(df_workday['day'])]#工作日IOS新增用户数量
#    df_filter = df[df['日期'].isin(df_workday['day'])]#工作日IOS新增用户数量

    df_Andr_filter=df_Andr_filter.iloc[::-1]#调整顺序
    df_Ios_filter=df_Ios_filter.iloc[::-1]#调整顺序
#   df_filter=df_filter.iloc[::-1]    
    
    bar=Bar("安卓与苹果客户端过去两周【新增用户】情况对比", width=1000, height=500,background_color='white')
    bar.add("安卓端新增用户", df_Andr_filter[u'日期'], df_Andr_filter['新增用户'], mark_point=['max'], mark_line=['average'],legend_text_color='red', is_datazoom_show=True)
    bar.add("IOS端新增用户", df_Ios_filter[u'日期'], df_Ios_filter['新增用户'], mark_point=['max'],mark_line=['average'], legend_text_color='blue' )
    bar.render('./Andr_VS_Ios_new.html')

    bar = Bar("安卓与苹果客户端过去两周【活跃用户】情况对比",  width=1000, height=500,background_color='white')
    bar.add("安卓端活跃用户", df_Andr_filter[u'日期'], df_Andr_filter['活跃用户'], mark_point=['max'], mark_line=['average'],legend_text_color='red', is_datazoom_show=True)
    bar.add("IOS端活跃用户", df_Ios_filter[u'日期'], df_Ios_filter['活跃用户'], mark_point=['max'],mark_line=['average'], legend_text_color='blue' )
    bar.render('./Andr_VS_Ios_acitve.html')

    bar = Bar("安卓与苹果客户端过去两周【累计用户】情况对比",  width=1000, height=500,background_color='white')
    bar.add("安卓端累计用户", df_Andr_filter[u'日期'], df_Andr_filter['累计用户'], mark_point=['max'], mark_line=['average'],legend_text_color='red', is_datazoom_show=True)
    bar.add("IOS端累计用户", df_Ios_filter[u'日期'], df_Ios_filter['累计用户'], mark_point=['max'],mark_line=['average'], legend_text_color='blue' )
    bar.render('./Andr_VS_Ios_all.html')

    bar = pyecharts.Bar("安卓与苹果客户端过去两周【启动次数】情况对比", width=1000, height=500,background_color='white')
    bar.add("安卓端启动次数", df_Andr_filter[u'日期'], df_Andr_filter['启动次数'], mark_point=['max'], mark_line=['average'],legend_text_color='red', is_datazoom_show=True)
    bar.add("IOS端启动次数", df_Ios_filter[u'日期'], df_Ios_filter['启动次数'], mark_point=['max'],mark_line=['average'], legend_text_color='blue' )
    bar.render('./Andr_VS_Ios_open.html')

    
#做两系统对比图
    return df_Ios_filter,df_Andr_filter

In [6]:
_doubleweek('/Users/rong/Desktop/尚云/双周汇报/历史数据/Android_天_应用趋势_20200703_20200717.csv',
            '/Users/rong/Desktop/尚云/双周汇报/历史数据/iPhone_天_应用趋势_20200703_20200717.csv')

(            日期  新增用户  活跃用户   启动次数   累计用户  新增独立用户  活跃独立用户
 14  2020-07-03    98  3822  15835  56512      98    3822
 11  2020-07-06   115  3981  17910  56717     115    3981
 10  2020-07-07    97  3904  16572  56814      97    3904
 9   2020-07-08    87  4021  17868  56901      87    4021
 8   2020-07-09    91  3955  17665  56992      91    3955
 7   2020-07-10    84  3691  15334  57076      84    3691
 4   2020-07-13    97  3973  17630  57277      97    3973
 3   2020-07-14   120  3988  17573  57397     120    3988
 2   2020-07-15    99  4021  17822  57496      99    4021
 1   2020-07-16    92  3940  17210  57588      92    3940
 0   2020-07-17    66  3656  15500  57654      66    3656,
             日期  新增用户  活跃用户   启动次数    累计用户  新增独立用户  活跃独立用户
 14  2020-07-03   236  8267  27631  131369     236    8267
 11  2020-07-06   258  8504  28588  131928     258    8504
 10  2020-07-07   270  8668  29315  132198     270    8668
 9   2020-07-08   245  8424  28209  132443     245    8424
 8   202

In [7]:
from pyecharts import Line
#from pyecharts import options as opts

In [8]:
#半月汇报--同比与环比增长
from pyecharts import Line
def _doubleweek_compare(data_A,data_I,data_A_before,data_I_before,data_A_last,data_I_last):
    df_Andr=pd.read_csv(data_A)
    df_Ios=pd.read_csv(data_I)
    df=df_Andr.copy()
    df['新增用户']=df_Andr['新增用户']+df_Ios['新增用户']
    df['活跃用户']=df_Andr['活跃用户']+df_Ios['活跃用户']
    df['累计用户']=df_Andr['累计用户']+df_Ios['累计用户']
    df['启动次数']=df_Andr['启动次数']+df_Ios['启动次数']
    
    e = pd.bdate_range('8/5/2018', '6/15/2022')
    df_workday=pd.DataFrame(e.format())
    df_workday.columns =['day']
    
    df_filter = df[df[u'日期'].isin(df_workday['day'])]#工作日安卓新增用户数量

    df_filter=df_filter.iloc[::-1]#调整顺序
#--------------------------------------------------------------上个两周
    df_Andr_before=pd.read_csv(data_A_before)
    df_Ios_before=pd.read_csv(data_I_before)
    df_before=df_Andr_before.copy()
    df_before['新增用户']=df_Andr_before['新增用户']+df_Ios_before['新增用户']
    df_before['活跃用户']=df_Andr_before['活跃用户']+df_Ios_before['活跃用户']
    df_before['累计用户']=df_Andr_before['累计用户']+df_Ios_before['累计用户']
    df_before['启动次数']=df_Andr_before['启动次数']+df_Ios_before['启动次数']
    
    df_filter_before = df_before[df_before['日期'].isin(df_workday['day'])]#工作日IOS新增用户数量

    df_filter_before=df_filter_before.iloc[::-1]#调整顺序
#----------------------------------------------------------去年同期

    df_Andr_last=pd.read_csv(data_A_last)
    df_Ios_last=pd.read_csv(data_I_last)
    df_last=df_Andr_last.copy()
    df_last['新增用户']=df_Andr_last['新增用户']+df_Ios_last['新增用户']
    df_last['活跃用户']=df_Andr_last['活跃用户']+df_Ios_last['活跃用户']
    df_last['累计用户']=df_Andr_last['累计用户']+df_Ios_last['累计用户']
    df_last['启动次数']=df_Andr_last['启动次数']+df_Ios_last['启动次数']
    
    df_filter_last = df_last[df_last['日期'].isin(df_workday['day'])]#工作日IOS新增用户数量


    df_filter_last=df_filter_last.iloc[::-1]#调整顺序
#----------------------------------------------------------    
    dataall_1=df_filter_before['累计用户']
    dataall_2=df_filter_last['累计用户']

    datanew_1=df_filter_before['新增用户']
    datanew_2=df_filter_last['新增用户']

    dataact_1=df_filter_before['活跃用户']
    dataact_2=df_filter_last['活跃用户']

    dataopen_1=df_filter_before['启动次数']
    dataopen_2=df_filter_last['启动次数']



    #做两周总量--累计用户，每日新增用户，活跃用户，启动次数对比图
    line=Line("累计用户同比/环比图", '2020/07/03-2020/07/17', width=1000, height=500)
    line.add('当前累计用户', df_filter['日期'], df_filter['累计用户'],mark_point=['max'], mark_line=['average'],is_datazoom_show=True,is_xaxis_boundarygap = True)
    line.add("去年同期累计用户", df_filter['日期'], dataall_1, mark_line=['average'], is_smooth=True,is_xaxis_boundarygap = True)
    line.add("两周前累计用户", df_filter['日期'], dataall_2, mark_line=['average'], is_smooth=True,is_xaxis_boundarygap = True)
    line.render('./compare_all.html')    
    
    line = Line("每日新增用户同比/环比图", '2020/07/03-2020/07/17',  width=1000, height=500)
    line.add("当前每日新增用户", df_filter['日期'], df_filter['新增用户'], mark_point=['max'],  mark_line=['average'],is_datazoom_show=True,is_xaxis_boundarygap = True)
    line.add("去年同期每日新增用户", df_filter['日期'], datanew_1, mark_line=['average'], is_smooth=True,is_xaxis_boundarygap = True)
    line.add("两周前每日新增用户", df_filter['日期'], datanew_2, mark_line=['average'], is_smooth=True,is_xaxis_boundarygap = True)
    #yaxis_opts=opts.AxisOpts(min_=minval2)
    line.render('./compare_new.html')

    line = Line("每日活跃用户同比/环比图", '2020/07/03-2020/07/17', width=1000, height=500)
    line.add("当前每日活跃用户", df_filter['日期'], df_filter['活跃用户'], mark_point=['max'],  mark_line=['average'],is_datazoom_show=True,is_xaxis_boundarygap = True)
    line.add("去年同期每日活跃用户", df_filter['日期'], dataact_1, mark_line=['average'], is_smooth=True,is_xaxis_boundarygap =True)
    line.add("两周前每日活跃用户", df_filter['日期'], dataact_2, mark_line=['average'], is_smooth=True,is_xaxis_boundarygap = True)
    #yaxis_opts=opts.AxisOpts(min_=minval3)
    line.render('./compare_active.html')

    line = Line("每日启动次数同比/环比图", '2020/07/03-2020/07/17', width=1000, height=500)
    line.add("当前每日启动次数", df_filter['日期'], df_filter['启动次数'], mark_point=['max'],  mark_line=['average'],is_datazoom_show=True,is_xaxis_boundarygap = True)
    line.add("去年同期每日启动次数", df_filter['日期'], dataopen_1, mark_line=['average'], is_smooth=True,is_xaxis_boundarygap = True)
    line.add("两周前每日启动次数", df_filter['日期'], dataopen_2, mark_line=['average'], is_smooth=True,is_xaxis_boundarygap =True)
    #yaxis_opts=opts.AxisOpts(min_=minval4)
    line.render('./compare_open.html')

    return df_filter_last['累计用户'],df_Andr_last['累计用户'],df_Ios_last['累计用户']

In [9]:
_doubleweek_compare('/Users/rong/Desktop/尚云/双周汇报/历史数据/Android_天_应用趋势_20200703_20200717.csv',
                    '/Users/rong/Desktop/尚云/双周汇报/历史数据/iphone_天_应用趋势_20200703_20200717.csv',
                    '/Users/rong/Desktop/尚云/双周汇报/历史数据/Android_天_应用趋势_20190703_20190717.csv',
                    '/Users/rong/Desktop/尚云/双周汇报/历史数据/iphone_天_应用趋势_20190703_20190717.csv',
                    '/Users/rong/Desktop/尚云/双周汇报/历史数据/Android_天_应用趋势_20200619_20200703.csv',
                    '/Users/rong/Desktop/尚云/双周汇报/历史数据/iphone_天_应用趋势_20200619_20200703.csv')

(14    183815
 11    184623
 10    184996
 9     185309
 8     185446
 7     185563
 4     186397
 3     186763
 2     187168
 1     187547
 0     187881
 Name: 累计用户, dtype: int64, 0     131369
 1     131133
 2     130849
 3     130555
 4     130301
 5     130027
 6     129796
 7     129681
 8     129600
 9     129490
 10    129258
 11    128986
 12    128665
 13    128537
 14    128387
 Name: 累计用户, dtype: int64, 0     56512
 1     56414
 2     56319
 3     56208
 4     56096
 5     55999
 6     55922
 7     55882
 8     55846
 9     55819
 10    55738
 11    55637
 12    55525
 13    55487
 14    55428
 Name: 累计用户, dtype: int64)

In [10]:
#半月汇报--两用户端用户留存
def _doubleweek_retain_Andr(data_A_1,data_A_2):
    df_1=pd.read_csv(data_A_1)
    df_2=pd.read_csv(data_A_2)
    df_1=df_1.drop('时间',axis=1)
    df_1=df_1.drop('新增用户',axis=1)
    df_2=df_2.drop('时间',axis=1)
    df_2=df_2.drop('新增用户',axis=1)

    sum1_o=pd.DataFrame(df_1.loc[1].copy())
    sum1=df_1.append(sum1_o,ignore_index=True)
    sum2=df_2.append(sum1_o,ignore_index=True)
    
    sum1['1天后留存'].loc[15]=df_1['1天后留存'].mean()
    sum1['2天后留存'].loc[15]=df_1['2天后留存'].mean()
    sum1['3天后留存'].loc[15]=df_1['3天后留存'].mean()
    sum1['4天后留存'].loc[15]=df_1['4天后留存'].mean()
    sum1['5天后留存'].loc[15]=df_1['5天后留存'].mean()
    sum1['6天后留存'].loc[15]=df_1['6天后留存'].mean()
    sum1['7天后留存'].loc[15]=df_1['7天后留存'].mean()
    sum1['14天后留存'].loc[15]=df_1['14天后留存'].mean()

    sum2['1天后留存'].loc[15]=df_2['1天后留存'].mean()
    sum2['2天后留存'].loc[15]=df_2['2天后留存'].mean()
    sum2['3天后留存'].loc[15]=df_2['3天后留存'].mean()
    sum2['4天后留存'].loc[15]=df_2['4天后留存'].mean()
    sum2['5天后留存'].loc[15]=df_2['5天后留存'].mean()
    sum2['6天后留存'].loc[15]=df_2['6天后留存'].mean()
    sum2['7天后留存'].loc[15]=df_2['7天后留存'].mean()
    sum2['14天后留存'].loc[15]=df_2['14天后留存'].mean()

    attr=['1天后留存','2天后留存','3天后留存','4天后留存','5天后留存','6天后留存','7天后留存','14天后留存']
    val1=[sum1['1天后留存'].loc[15],sum1['2天后留存'].loc[15],sum1['3天后留存'].loc[15],sum1['4天后留存'].loc[15],sum1['5天后留存'].loc[15],sum1['6天后留存'].loc[15],sum1['7天后留存'].loc[15],sum1['14天后留存'].loc[15]]
    val2=[sum2['1天后留存'].loc[15],sum2['2天后留存'].loc[15],sum2['3天后留存'].loc[15],sum2['4天后留存'].loc[15],sum2['5天后留存'].loc[15],sum2['6天后留存'].loc[15],sum2['7天后留存'].loc[15],sum2['14天后留存'].loc[15]]
    
    from pyecharts import Line
    line=Line('用户留存分析',width=1000, height=500,background_color='white')
    line.add('安卓端当前周平均留存率',attr,val1, mark_point=['max'],legend_text_color='red', is_datazoom_show=True,is_xaxis_boundarygap = True)
    line.add('安卓端之前周平均留存率',attr,val2, mark_point=['max'],legend_text_color='red', is_datazoom_show=True,is_xaxis_boundarygap = True)
    line.render('./Andr_retain.html')
#做两系统对比图
    return

def _doubleweek_retain_Ios(data_A_1,data_A_2):
    df_1=pd.read_csv(data_A_1)
    df_2=pd.read_csv(data_A_2)
    df_1=df_1.drop('时间',axis=1)
    df_1=df_1.drop('新增用户',axis=1)
    df_2=df_2.drop('时间',axis=1)
    df_2=df_2.drop('新增用户',axis=1)

    sum1_o=pd.DataFrame(df_1.loc[1].copy())
    sum1=df_1.append(sum1_o,ignore_index=True)
    sum2=df_2.append(sum1_o,ignore_index=True)
    
    sum1['1天后留存'].loc[15]=df_1['1天后留存'].mean()
    sum1['2天后留存'].loc[15]=df_1['2天后留存'].mean()
    sum1['3天后留存'].loc[15]=df_1['3天后留存'].mean()
    sum1['4天后留存'].loc[15]=df_1['4天后留存'].mean()
    sum1['5天后留存'].loc[15]=df_1['5天后留存'].mean()
    sum1['6天后留存'].loc[15]=df_1['6天后留存'].mean()
    sum1['7天后留存'].loc[15]=df_1['7天后留存'].mean()
    sum1['14天后留存'].loc[15]=df_1['14天后留存'].mean()

    sum2['1天后留存'].loc[15]=df_2['1天后留存'].mean()
    sum2['2天后留存'].loc[15]=df_2['2天后留存'].mean()
    sum2['3天后留存'].loc[15]=df_2['3天后留存'].mean()
    sum2['4天后留存'].loc[15]=df_2['4天后留存'].mean()
    sum2['5天后留存'].loc[15]=df_2['5天后留存'].mean()
    sum2['6天后留存'].loc[15]=df_2['6天后留存'].mean()
    sum2['7天后留存'].loc[15]=df_2['7天后留存'].mean()
    sum2['14天后留存'].loc[15]=df_2['14天后留存'].mean()

    attr=['1天后留存','2天后留存','3天后留存','4天后留存','5天后留存','6天后留存','7天后留存','14天后留存']
    val1=[sum1['1天后留存'].loc[15],sum1['2天后留存'].loc[15],sum1['3天后留存'].loc[15],sum1['4天后留存'].loc[15],sum1['5天后留存'].loc[15],sum1['6天后留存'].loc[15],sum1['7天后留存'].loc[15],sum1['14天后留存'].loc[15]]
    val2=[sum2['1天后留存'].loc[15],sum2['2天后留存'].loc[15],sum2['3天后留存'].loc[15],sum2['4天后留存'].loc[15],sum2['5天后留存'].loc[15],sum2['6天后留存'].loc[15],sum2['7天后留存'].loc[15],sum2['14天后留存'].loc[15]]
    
    from pyecharts import Line
    line=Line('用户留存分析',width=1000, height=500,background_color='white')
    line.add('Ios端当前周平均留存率',attr,val1, mark_point=['max'],legend_text_color='red', is_datazoom_show=True,is_xaxis_boundarygap = True)
    line.add('Ios端之前周平均留存率',attr,val2, mark_point=['max'],legend_text_color='red', is_datazoom_show=True,is_xaxis_boundarygap = True)
    line.render('./Ios_retain.html')
#做两系统对比图
    return

In [11]:
_doubleweek_retain_Andr("/Users/rong/Desktop/尚云/双周汇报/历史数据/环评云助手Android_天_新增用户留存_留存用户_率_20200703_20200717.csv",
 "/Users/rong/Desktop/尚云/双周汇报/历史数据/环评云助手Android_天_新增用户留存_留存用户_率_20200619_20200703.csv")

In [12]:
_doubleweek_retain_Ios("/Users/rong/Desktop/尚云/双周汇报/历史数据/环评云助手iOS_天_新增用户留存_留存用户_率_20200703_20200717.csv",
 "/Users/rong/Desktop/尚云/双周汇报/历史数据/环评云助手iOS_天_新增用户留存_留存用户_率_20200619_20200703.csv")